In [35]:
from tqdm import tqdm
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.representation import MaximalMarginalRelevance # 导入主题展示优化模型
from sklearn.cluster import KMeans # 导入Kmeans

# 切词txt文件
cut_word_file_path = '../数据文件/切词数据/切词-论文数据.txt'
# 模型名称
model_name = "aspire/acge_text_embedding"
# 本地的词向量路径
emb_file_path = '../数据文件/词向量/论文-acge_text_embedding.npy'
# 二次停用词，可以二次传入停用词，将文本中的出现次数太高且无区分意义的词去掉
second_stopwords_list = open('../stopwords/second_stopwords.txt', 'r', encoding='utf-8').read().split('\n')

# 加载分词文件和向量文件

In [36]:
# step1 加载文件
with open(cut_word_file_path, 'r', encoding='utf-8') as file:
  docs = file.readlines()
print(f'条数: {len(docs)}，预览第一条：{docs[0]}')

# step2 加载向量文件
embeddings = np.load(emb_file_path)
print(embeddings.shape)

条数: 24651，预览第一条：附赠 AI 只为 拥有 服务

(24651, 1792)


# UMAP超参数设置

In [37]:
# n_neighbors：通常在5到50之间，表示用于构建初始高维图的近似最近邻点的数量。较小的值更关注局部结构，较大的值更关注全局结构。
n_neighbors=10
# n_components：通常为2，表示降维后的目标维数。你可以选择其他值，但2维通常足够用于可视化和分析。
n_components=5
# min_dist：通常在0.0到1.0之间，控制嵌入的紧密程度。较小的值会使数据点更紧密地聚集在低维空间中，但可能导致过拟合。
min_dist=0.0
# metric：参数决定了在输入空间中测量距离时使用的度量方式。'cosine'余弦相似度,'manhattan'曼哈顿距离,'euclidean'欧氏距离
metric='cosine'
# random_state：随机数种子
random_state=42

# HDBSCAN超参数设置

In [38]:
# HDBSCAN聚类中，每个簇至少包含多少个文档
min_cluster_size=100
# 此参数越大，聚类结果越保守，离群值越多；此参数默认与min_cluster_size保持一致，但也可以手动调节。
min_samples=10

# 设置模型

In [39]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model=model_name) # 使用对应的向量模型

# 2. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=n_neighbors,
  n_components=n_components, #将高维向量降维到n维
  min_dist=min_dist, # ⚠️ 该参数数值越小，聚类越密集。一般设置0.1或0.0左右。
  metric=metric,
  random_state=random_state  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 3. 创建HDBSCAN聚类模型
hdbscan_model = HDBSCAN(
    min_cluster_size=min_cluster_size,  #每个聚类中，至少包括n个文档
    min_samples=min_samples,  #此参数越大，聚类结果越保守，离群值越多；此参数默认与min_cluster_size保持一致，但也可以手动调节。
    prediction_data = True
    )

# 3. 创建K-means聚类模型（或者使用K-means聚类）
# cluster_model = KMeans(n_clusters=6) # 要聚成几个类 ⚠️ n_clusters参数需要相对应的方法来确定

# 4. 创建分词模型
vectorizer_model = CountVectorizer(
   stop_words=second_stopwords_list  # 可以二次传入停用词，将文本中的出现次数太高且无区分意义的词去掉
)                             #因为我们已经分好词了，所以这里不需要传入分词函数了

# 5. 主题展示优化模型，主题词会更多样化，减少冗余词
# diversity: How diverse the select keywords/keyphrases are.
#                     Values range between 0 and 1 with 0 being not diverse at all
#                     and 1 being most diverse.
representation_model = MaximalMarginalRelevance(diversity=0.3) # 创建mmr模型

# 训练主题模型

In [40]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  hdbscan_model=hdbscan_model,                # 若使用K-means聚类模型，则参数改为对应的模型变量
  umap_model=umap_model,
  representation_model=representation_model   # 传入主题展示优化模型
  #, min_topic_size=40  # 设置每个聚类至少又要N个文档。⚠️若在hdbscan模型中已经设置了min_cluster_size参数，则无需设置此参数。
  #, nr_topics=5        # 该参数会指定聚类结果为n个主题
)
# 训练模型
topic_model = topic_model.fit(docs, embeddings)
# 保存模型
topic_model.save("./0_topic_model")

# 查看主题信息
topic_info = topic_model.get_topic_info()
topic_info

2024-07-28 22:11:27,761 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


,Topic,Count,Name,Representation,Representative_Docs
0,-1,10825,-1_科技_未来_机器人_创造,"[科技, 未来, 机器人, 创造, 社会, 知识, 答案, 生命, 训练, 时代]",[脑机 接口 成熟 模型 ai 无线连接 人类 知识 越来越 普通人 学到 领域 边界 时间...
1,0,1635,0_gpt_模型_chat_训练,"[gpt, 模型, chat, 训练, 答案, gpt4, 论文, 代码, 生成, 搜索]",[AI 语言 生成 模型 GPT 确实 发展 能力 自然语言 生成 文本 分类 问答 突破 ...
2,1,1405,1_答案_观点_正确_时代,"[答案, 观点, 正确, 时代, 训练, 这句, 淘汰, 逻辑, 真人, 背错]","[有没有 思想 正确\n, 简单 纠正 正确 答案\n, 纠正 正确 答案\n]"
3,2,1391,2_论文_老师_作业_水课,"[论文, 老师, 作业, 水课, 毕业论文, 查重, 研究生, 作文, 搜题, 网课]","[论文\n, 论文 东西\n, 论文\n]"
4,3,1211,3_百度_bing_广告_搜索引擎,"[百度, bing, 广告, 搜索引擎, 搜索, 谷歌, newbing, google, ...","[百度\n, 反正 百度 搜索 广告 广告 搜索引擎\n, 百度 广告\n]"
5,4,557,4_画师_美术_作品_绘图,"[画师, 美术, 作品, 绘图, 画家, 美工, 作画, 原画, 画手, 画风]",[ai 取代 画师 画师 行业 影响 网民 阴阳怪气 抽象 攻击 目标 画师 群体 ai 绘...
6,5,437,5_消灭_进化_宇宙_生物,"[消灭, 进化, 宇宙, 生物, 智慧, 人脑, 人类文明, 未来, 算力, 机器人]","[地球 未来 ai 回答 消灭 人类 可怕\n, ai 人类 消灭 人类\n, ai 人类 ..."
7,6,398,6_微软_windows_小冰_linux,"[微软, windows, 小冰, linux, 操作系统, 盗版, office, 软件,...","[微软\n, 微软\n, 微软\n]"
8,7,381,7_失业_工资_打工_裁员,"[失业, 工资, 打工, 裁员, 套路, 加班, 就业, 工作岗位, 中层, 保安]","[失业\n, 失业\n, 老板 工作 相关\n]"
9,8,367,8_生产力_生产关系_工业革命_社会,"[生产力, 生产关系, 工业革命, 社会, 失业, 矛盾, 人口, 工业, 共产主义, 剥削]",[生产力 提升 生产关系 适配 生产力 生产力 还会 生产关系 改变 至少 人类 发展 数千...
